In [11]:
from datetime import date,datetime
import json
import csv
import time
from bs4 import BeautifulSoup
import requests
from dataclasses import dataclass


@dataclass
class Holiday:
    name: str
    date: datetime
    tag: str

    def __str__ (self):
        return self.name + " (" + self.date.strftime("%Y-%m-%d") + ")"

@dataclass
class HolidayList:
    holidays: list

    def addHoliday(self):
        
        print("Add a Holiday")
        print("=============\n")

        print("Holiday: ")

        time.sleep(1)
        name = input()
        while(True):
            print("Date (example: Jan 1 2021): ")

            time.sleep(1)
            Date = input()

            try:
                Date = datetime.strptime(Date,"%b %d %Y")
                break
            except:
                print("Error: invaild date format")
                continue

        print("Tag: ")

        time.sleep(1)
        tag = input()

        temp = Holiday(name,Date,tag)

        print("\nSuccess:")
        print(temp)
        print("has been added to the holiday list.\n")
        
        self.holidays.append(temp)

    def removeHoliday(self):
        
        print("Remove a Holiday")
        print("================\n")
        while(True):
            print("Holiday Name: ")

            time.sleep(1)
            name = input()

            
            temp = self.numHolidays()

            self.holidays = [x for x in self.holidays if x.name != name]

            if temp > self.numHolidays():
                print("\nSuccess:")
                print(name)
                print("has been removed from the holiday list.\n")
                break
            else:
                print("Error: holiday not found")
                continue

    def read_json(self,filelocation):
        
        try:
            with open(filelocation,"r") as file:
                load = json.load(file)
                
                for x in load["holidays"]:
                    self.holidays.append(Holiday(x["name"],(datetime.strptime(x["date"], "%b %d %Y")).date(),x["tag"]))
        except:
            print("404")

    def save_to_json(self,filename):

        temp_list = []

        with open("Data/" + filename + ".json", "w") as file:

            for x in self.holidays:
                
                temp_list.append(x.__dict__)
            
            temp_dict = {"holidays": temp_list}

            json.dump(temp_dict,file,indent = 4, default = str)
                

    def save_to_csv(self,filename):
        
        with open("Data/" + filename + ".csv","w",newline="",encoding="utf-8") as file:

            writer = csv.writer(file)
            writer.writerow(self.holidays[0].__dict__)
            for x in range(0,len(self.holidays)):
                writer.writerow(self.holidays[x].__dict__.values())

    def save(self,filelocation):

        print("Saving Holiday List")
        print("====================\n")

        print("Are you sure you want to save your changes? [y/n]:")

        time.sleep(1)
        y_n = input()

        if y_n == "y":
            while(True):
                print("\nwould you like it saved to json or csv? [json/csv]:")

                time.sleep(1)
                json_csv = input().lower()

                if json_csv == "json":
                    self.save_to_json(filelocation)
                    break
                elif json_csv == "csv":
                    self.save_to_csv(filelocation)
                    break
                else: 
                    print("invalid input\n")
                    continue

        else:
            print("Canceled:\nHoliday list file save canceled.")

    def tag_search(self):
        pass

    def date_search(self):

        print("Date Search Holiday List")
        print("========================\n")

        flag = True
        print("Date? (example: 2020-01-14): ")

        time.sleep(1)
        date = input()

        for x in self.holidays:
            if x.date.strftime("%Y-%m-%d") == date:
                flag = False
                print("\n",x)
            
        if flag:
            print("No Holidays Found")
                
    def numHolidays(self):
        return len(self.holidays)

    def scrapeHolidays(self):

        todays_date = date.today()

        counter = todays_date.year - 2

        while counter < todays_date.year + 3:

            try:
                fetchedData = requests.get("https://www.timeanddate.com/holidays/us/"+ str(counter) +"?hol=9565233").text

                soup = BeautifulSoup(fetchedData, "lxml")

                data = soup.find_all("tr", class_= "showrow")

                for holiday in data:

                    name = holiday.find("a").text
                    tag = holiday.find_all("td")[2].get_text()
                    holiday_date = datetime.strptime(holiday.find("th").text + " " +str(counter), "%b %d %Y")

                    self.holidays.append(Holiday(name,holiday_date.date(),tag))
            except:
                print("can't reach website")   

            counter += 1

def main():

    holiday_list = HolidayList([])
    holiday_list.read_json("Data/holidays.json")

    if len(holiday_list.holidays) < 8:
        holiday_list.scrapeHolidays()

    while(True):

        menuselection = 0
        
        print("\nHoliday Menu")
        print("================")
        print("1. Add a Holiday")
        print("2. Remove a Holiday")
        print("3. Save Holiday List")
        print("4. View Holidays")
        print("5. Tag Search")
        print("6. Date Search")
        print("7. Exit\n")

        time.sleep(1)
        menuselection = input()

        if menuselection == "1":
            holiday_list.addHoliday()
        elif menuselection == "2":
            holiday_list.removeHoliday()
        elif menuselection == "3":
            print("What would you like the file to be named?")

            time.sleep(1)
            filename = input()

            holiday_list.save(filename)
        elif menuselection == "4":
            pass
        elif menuselection == "5":
            holiday_list.tag_search()
        elif menuselection == "6":
            holiday_list.date_search()
        elif menuselection == "7":
            break
        else: print("Not Vaild Input\n")

    #for x in holiday_list.holidays:
       #print(x)

    print("Goodbye")

if __name__ == "__main__":
    main()



Holiday Menu
1. Add a Holiday
2. Remove a Holiday
3. Save Holiday List
4. View Holidays
5. Tag Search
6. Date Search
7. Exit

Date Search Holiday List

Date? (example: 2020-01-14): 
No Holidays Found

Holiday Menu
1. Add a Holiday
2. Remove a Holiday
3. Save Holiday List
4. View Holidays
5. Tag Search
6. Date Search
7. Exit

Date Search Holiday List

Date? (example: 2020-01-14): 
Mother's Day (2020-05-10)

Holiday Menu
1. Add a Holiday
2. Remove a Holiday
3. Save Holiday List
4. View Holidays
5. Tag Search
6. Date Search
7. Exit

Goodbye
